In [16]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import jenkspy
import matplotlib.pyplot as plt

In [2]:
# clip ookla data to eu wide
ookla = gpd.read_file(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\ookla\2024-01-01_performance_mobile_tiles\gps_mobile_tiles.shp')
ookla.to_crs('EPSG:3035')
mask_clip = gpd.clip(ookla, nuts)
ookla_eu = mask_clip[~mask_clip.is_empty]
ookla_eu.to_file(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\ookla\2024-01-01_performance_mobile_tiles\ookla_eu.shp')

In [5]:
ookla_eu = gpd.read_file(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\ookla\2024-01-01_performance_mobile_tiles\ookla_24_1qr_clip.shp')
ookla_eu.to_crs('EPSG:3035')
# extract ookla polygon to centroids
ookla_pt = ookla_eu.copy()
ookla_pt.geometry = ookla_pt['geometry'].centroid
ookla_pt.crs = ookla_eu.crs
ookla_pt.to_file(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\ookla\2024-01-01_performance_mobile_tiles\ookla_24_1qr_eu_pt.shp')

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,Shape_Leng,Shape_Area,geometry
0,0313102133033211,1700,6510,24,1,1,1458.165163,132888.452032,POINT (3098330.310 3529520.980)
1,0311332332313212,22995,17377,29,3,1,1375.362007,118225.164146,POINT (3499119.563 3712302.537)
2,0313103021112001,126381,41120,21,2,1,1448.788408,131185.197566,POINT (3158606.333 3545553.113)
3,0313101320101320,2310,1518,67,1,1,1420.165778,126053.037175,POINT (3245822.618 3619699.790)
4,0310131111231302,553,333,44,1,1,1073.146253,71949.994197,POINT (2783214.026 4917639.435)


In [17]:
nuts = gpd.read_file(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\NUTS\NUTS_RG_01M_2021_3035\NUTS_RG_01M_2021_3035.shp')
ookla_pt = gpd.read_file(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\0-data\ookla\2024-01-01_performance_mobile_tiles\ookla_24_1qr_eu_pt.shp')

In [18]:
speed_dict = {}
for nuts_id in nuts['NUTS_ID']:
    area = nuts[nuts['NUTS_ID']==nuts_id]
    ookla_within = gpd.sjoin(ookla_pt, area, how='inner')
    ookla_within['area*d'] = ookla_within['avg_d_kbps']*ookla_within['Shape_Area']
    ookla_within['area*u'] = ookla_within['avg_u_kbps']*ookla_within['Shape_Area']
    avg_d_kpbs = ookla_within['area*d'].sum()/ookla_within['Shape_Area'].sum()
    avg_u_kbps = ookla_within['area*u'].sum()/ookla_within['Shape_Area'].sum()
    speed_dict[nuts_id] = ['kbps', avg_d_kpbs, avg_u_kbps, '2024-1st-qtr']

In [19]:
speed_df = pd.DataFrame(speed_dict).T
speed_df.reset_index(inplace=True)
speed_df.columns = ['geo', 'unit', 'avg_d_kbps', 'avg_u_kbps', 'obsTime']
speed_df.to_excel(r'D:\OneDrive - Nexus365\1-Oxford\1 - Oxford Uni related\7 - Services\2024.05.20 Mapineq\2-project\try.xlsx')